# Task & Talk
Full reimplementation of the task in [Kottur, et al. (2017)](https://arxiv.org/pdf/1706.08502.pdf). Using tabular Q-learning for now.

## Initialization and Housekeeping
Set parameters, initialize the Q-tables, and describe any helper functions we might need for post-analysis.
### Parameters

In [127]:
num_episodes = 100000
eta = 0.8
gamma = 0.95

epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.0001;

### Setting up Q-Table

In [128]:
q_vocab = 3
a_vocab = 12

num_tasks = 6
num_attributes = 4;

In [129]:
# A-State: [att * att, q_vocab, a_vocab+1, q_vocab+1] (+1 for empty vocab)
# Q-State (Utterance): [task, q_vocab+1, a_vocab+1] (+1 for empty vocab)
# Q-State (Guess): [task, q_vocab, a_vocab, q_vocab, a_vocab]
a_table = zeros(num_attributes, num_attributes, q_vocab, a_vocab+1, q_vocab+1, a_vocab)
q_table_utt = zeros(num_tasks, q_vocab+1, a_vocab+1, q_vocab)
q_table_guess = zeros(num_tasks, q_vocab, a_vocab, q_vocab, a_vocab, num_attributes, num_attributes)

a_visited = falses(num_attributes, num_attributes, q_vocab, a_vocab+1, q_vocab+1)
q_utt_visited = falses(num_tasks, q_vocab+1, a_vocab+1)
q_guess_visited = falses(num_tasks, q_vocab, a_vocab, q_vocab, a_vocab)

num_correct = 0;

## Accuracy measurements

In [130]:
num_exploits = 0;

### Helper Functions

In [131]:
function get_reward(a_state, guess)
    if a_state[1:2] == guess
        return 1
    else
        return 0
    end
end

get_reward (generic function with 1 method)

## Q-Learning
The core loop of the program:

In [ ]:
total_rewards = 0
for episode in 1:num_episodes
    tradeoff = rand() # exploration-exploitation
    explore = (tradeoff < epsilon)
    
    # Generate random task:
    q_state = [rand(1:num_tasks), q_vocab+1, a_vocab+1]
    q_states = [copy(q_state)]
    
    
    # Q-BOT
    #  -> TURN 1
    if explore | ~q_utt_visited[q_state...]
        # Explore:
        q_state[2] = rand(1:q_vocab)
    else
        # Exploit:
        options = q_table_utt[q_state[1], q_state[2], q_state[3], :]
        q_state[2] = argmax(options)
    end
    
    # Generate random object:
    a_state = [rand(1:num_attributes), rand(1:num_attributes), q_state[2], a_vocab+1, q_vocab+1]
    a_states = [copy(a_state)]
    
    
    # A-BOT
    #  -> TURN 1
    if explore | ~a_visited[a_state...]
        # Explore:
        a_state[4] = rand(1:a_vocab)
    else
        # Exploit:
        options = a_table[a_state[1], a_state[2], a_state[3], a_state[4], a_state[5], :]
        a_state[4] = argmax(options)
    end
    
    q_state[3] = a_state[4]
    push!(q_states, copy(q_state))
    
    
    # Q-BOT
    #  -> TURN 2
    if explore | ~q_utt_visited[q_state...]
        # Explore:
        a_state[5] = rand(1:q_vocab)
    else
        # Exploit:
        options = q_table_utt[q_state[1], q_state[2], q_state[3], :]
        a_state[5] = argmax(options)
    end
    push!(a_states, copy(a_state))
    
    # Update Q-State for guessing attributes:
    q_state = [q_state[1], q_state[2], q_state[3], a_state[5], a_vocab+1]
    
    
    # A-BOT
    #  -> TURN 2
    if explore | ~a_visited[a_state...]
        # Explore:
        q_state[5] = rand(1:a_vocab)
    else
        # Exploit:
        options = a_table[a_state[1], a_state[2], a_state[3], a_state[4], a_state[5], :]
        q_state[5] = argmax(options)
    end
    
    
    # Q-BOT
    #  -> GUESSING PHASE
    guess = []
    if explore | ~q_guess_visited[q_state...]
        # Explore:
        guess = [rand(1:num_attributes), rand(1:num_attributes)]
    else
        # Exploit:
        options = q_table_guess[q_state[1], q_state[2], q_state[3], q_state[4], q_state[5], :, :]
        optimal_first = num_attributes + 1 # Running best attribute
        optimal_second = num_attributes + 1
        max_val = -99999
        for att1 in 1:num_attributes
            for att2 in 1:num_attributes
                if (options[att1, att2] > max_val)
                    max_val = options[att1, att2]
                    optimal_first = att1
                    optimal_second = att2
                end
            end
        end
        guess = [optimal_first, optimal_second]
    end
    
    # Update Reward Tables:
    reward = get_reward(a_state, guess)
    
    a_table[a_state[1],a_state[2],a_state[3],a_vocab+1, q_vocab+1, q_state[3]] += reward
    a_visited[a_state[1],a_state[2],a_state[3],a_vocab+1, q_vocab+1] = true
    a_table[a_state[1],a_state[2],a_state[3],a_state[4],a_state[5], q_state[5]] += reward
    a_visited[a_state[1],a_state[2],a_state[3],a_state[4],a_state[5]] = true
    
    q_table_utt[q_state[1],q_vocab+1, a_vocab+1, q_state[2]] += reward
    q_utt_visited[q_state[1],q_vocab+1, a_vocab+1] = true
    q_table_utt[q_state[1],q_state[2],q_state[3], q_state[4]] += reward
    q_utt_visited[q_state[1],q_state[2],q_state[3]] = true
    
    q_table_guess[q_state[1],q_state[2],q_state[3],q_state[4],q_state[5], guess[1], guess[2]] += reward
    q_guess_visited[q_state[1],q_state[2],q_state[3],q_state[4],q_state[5]] = true

    epsilon = min_epsilon + (max_epsilon - min_epsilon)*exp(-decay_rate*episode)
    if (~explore) & (episode > 50000)
        println(a_state)
        println(guess)
        total_rewards += reward
        num_exploits += 1
    end
end
print(num_exploits)
println("Accuracy: ", total_rewards/num_exploits)

[1, 4, 3, 6, 3]
[1, 4]
[2, 3, 1, 4, 1]
[1, 1]
[2, 4, 2, 2, 1]
[2, 4]
[4, 3, 1, 12, 3]
[1, 1]
[3, 3, 3, 7, 2]
[3, 3]
[2, 1, 3, 1, 3]
[3, 1]
[4, 2, 3, 3, 1]
[1, 1]
[1, 4, 2, 3, 1]
[1, 2]
[4, 4, 3, 12, 1]
[2, 3]
[2, 3, 1, 4, 3]
[1, 1]
[2, 2, 3, 9, 1]
[1, 1]
[1, 3, 3, 6, 3]
[1, 3]
[3, 3, 2, 1, 1]
[3, 3]
[2, 3, 1, 4, 3]
[1, 1]
[3, 4, 2, 9, 1]
[1, 1]
[1, 3, 2, 8, 2]
[1, 3]
[4, 4, 2, 3, 1]
[1, 1]
[4, 3, 2, 2, 1]
[1, 1]
[4, 2, 1, 5, 2]
[4, 2]
[3, 3, 1, 1, 3]
[1, 1]
[1, 3, 1, 3, 3]
[1, 1]
[4, 2, 3, 3, 1]
[1, 1]
[4, 2, 2, 2, 3]
[4, 2]
[2, 3, 2, 2, 3]
[2, 3]
[1, 4, 3, 6, 1]
[1, 1]
[4, 3, 1, 12, 1]
[4, 3]
[4, 1, 3, 7, 2]
[1, 1]
[1, 2, 1, 7, 3]
[1, 2]
[2, 3, 1, 4, 1]
[1, 1]
[3, 4, 2, 9, 2]
[3, 4]
[3, 2, 1, 3, 3]
[1, 1]
[2, 4, 3, 1, 3]
[1, 1]
[3, 2, 3, 3, 1]
[1, 1]
[4, 1, 2, 1, 1]
[1, 1]
[4, 4, 1, 1, 1]
[1, 1]
[3, 1, 1, 3, 3]
[1, 1]
[1, 2, 3, 3, 1]
[1, 1]
[4, 2, 2, 2, 1]
[1, 1]
[4, 3, 3, 10, 1]
[4, 3]
[4, 4, 1, 1, 3]
[1, 1]
[1, 4, 1, 4, 3]
[1, 4]
[4, 3, 2, 2, 3]
[2, 3]
[1, 4, 1, 4, 1]
[1, 1]
[1, 4, 

[1, 4, 2, 3, 1]
[1, 2]
[4, 2, 3, 3, 1]
[1, 1]
[1, 2, 1, 7, 3]
[1, 2]
[1, 4, 1, 4, 1]
[1, 1]
[1, 3, 1, 3, 2]
[1, 3]
[2, 4, 3, 1, 3]
[2, 4]
[3, 2, 1, 3, 2]
[1, 1]
[4, 4, 3, 12, 2]
[4, 4]
[2, 4, 3, 1, 3]
[1, 1]
[1, 3, 1, 3, 2]
[1, 3]
[3, 4, 3, 10, 2]
[1, 2]
[3, 2, 1, 3, 3]
[1, 1]
[3, 4, 3, 10, 1]
[1, 2]
[2, 4, 3, 1, 3]
[1, 1]
[4, 2, 1, 5, 1]
[1, 1]
[4, 1, 3, 7, 2]
[1, 1]
[2, 2, 3, 9, 1]
[1, 1]
[1, 1, 1, 1, 1]
[1, 1]
[2, 1, 2, 11, 1]
[2, 1]
[2, 4, 3, 1, 3]
[2, 4]
[4, 2, 2, 2, 3]
[4, 2]
[4, 3, 1, 12, 3]
[1, 1]
[4, 1, 3, 7, 1]
[4, 1]
[2, 2, 1, 3, 3]
[2, 2]
[3, 2, 3, 3, 1]
[3, 2]
[4, 1, 1, 10, 1]
[4, 1]
[4, 3, 2, 2, 1]
[1, 1]
[3, 4, 2, 9, 2]
[3, 4]
[4, 1, 1, 10, 3]
[4, 1]
[2, 4, 3, 1, 3]
[2, 4]
[1, 4, 2, 3, 2]
[1, 1]
[3, 2, 3, 3, 1]
[3, 2]
[3, 1, 2, 6, 1]
[1, 1]
[1, 1, 1, 1, 3]
[1, 1]
[4, 4, 2, 3, 1]
[1, 1]
[3, 1, 3, 1, 3]
[2, 4]
[3, 1, 2, 6, 1]
[1, 1]
[1, 1, 2, 1, 1]
[1, 1]
[1, 4, 1, 4, 3]
[1, 4]
[4, 3, 3, 10, 1]
[4, 3]
[4, 4, 2, 3, 1]
[1, 1]
[3, 4, 1, 9, 3]
[3, 4]
[3, 3, 1, 1, 3]
[1, 1]
[3,

[2, 4]
[2, 4, 2, 2, 3]
[2, 3]
[2, 4, 1, 7, 3]
[1, 1]
[3, 2, 3, 3, 1]
[3, 2]
[4, 2, 1, 5, 2]
[4, 2]
[3, 3, 1, 1, 3]
[1, 1]
[4, 1, 3, 7, 2]
[1, 1]
[1, 4, 3, 6, 3]
[1, 4]
[3, 2, 1, 3, 3]
[1, 1]
[4, 2, 1, 5, 1]
[1, 1]
[3, 2, 2, 9, 2]
[1, 1]
[3, 2, 3, 3, 1]
[3, 2]
[3, 3, 3, 7, 1]
[4, 1]
[1, 3, 1, 3, 2]
[1, 3]
[2, 3, 3, 1, 3]
[1, 1]
[3, 1, 2, 6, 1]
[1, 1]
[4, 1, 1, 10, 3]
[4, 1]
[4, 1, 1, 10, 3]
[4, 1]
[3, 4, 1, 9, 2]
[1, 1]
[1, 2, 3, 3, 1]
[1, 1]
[4, 2, 1, 5, 1]
[1, 1]
[2, 3, 3, 1, 3]
[1, 1]
[2, 2, 1, 3, 2]
[1, 1]
[1, 3, 1, 3, 3]
[1, 1]
[2, 3, 1, 4, 3]
[1, 1]
[3, 1, 1, 3, 3]
[1, 1]
[2, 3, 2, 2, 1]
[1, 1]
[4, 1, 3, 7, 1]
[4, 1]
[1, 4, 2, 3, 1]
[1, 2]
[1, 4, 3, 6, 1]
[1, 1]
[1, 4, 1, 4, 3]
[1, 4]
[3, 2, 3, 3, 1]
[1, 1]
[1, 4, 3, 6, 1]
[1, 1]
[1, 2, 3, 3, 1]
[1, 1]
[2, 4, 2, 2, 1]
[2, 4]
[1, 3, 1, 3, 3]
[1, 1]
[4, 1, 3, 7, 1]
[4, 1]
[1, 2, 1, 7, 1]
[1, 1]
[1, 3, 1, 3, 2]
[1, 3]
[1, 2, 1, 7, 1]
[1, 1]
[2, 3, 2, 2, 3]
[2, 3]
[2, 1, 2, 11, 1]
[2, 1]
[3, 2, 1, 3, 3]
[1, 1]
[3, 4, 2, 9, 2]
[3, 4]
[

[3, 1, 1, 3, 3]
[1, 1]
[4, 1, 3, 7, 1]
[4, 1]
[4, 1, 3, 7, 1]
[4, 1]
[2, 3, 1, 4, 1]
[1, 1]
[1, 4, 2, 3, 1]
[1, 2]
[3, 3, 3, 7, 1]
[4, 1]
[1, 1, 1, 1, 1]
[1, 1]
[4, 4, 3, 12, 1]
[2, 3]
[4, 2, 2, 2, 1]
[1, 1]
[1, 1, 2, 1, 1]
[1, 1]
[4, 2, 1, 5, 1]
[1, 1]
[3, 2, 3, 3, 1]
[3, 2]
[1, 3, 1, 3, 2]
[1, 3]
[4, 2, 2, 2, 1]
[1, 1]
[4, 3, 2, 2, 1]
[1, 1]
[4, 3, 3, 10, 2]
[4, 3]
[2, 4, 2, 2, 3]
[2, 3]
[4, 3, 2, 2, 3]
[2, 3]
[1, 1, 1, 1, 1]
[1, 1]
[4, 1, 2, 1, 1]
[4, 1]
[3, 1, 2, 6, 1]
[3, 1]
[3, 3, 1, 1, 3]
[1, 1]
[4, 2, 1, 5, 2]
[4, 2]
[2, 2, 3, 9, 1]
[1, 1]
[3, 1, 2, 6, 1]
[1, 1]
[3, 1, 2, 6, 1]
[1, 1]
[1, 4, 1, 4, 3]
[1, 4]
[2, 3, 1, 4, 1]
[1, 1]
[1, 2, 2, 3, 1]
[1, 2]
[3, 4, 1, 9, 2]
[1, 1]
[4, 3, 1, 12, 3]
[1, 1]
[2, 4, 3, 1, 3]
[1, 1]
[1, 4, 1, 4, 1]
[1, 1]
[3, 3, 1, 1, 3]
[1, 1]
[2, 3, 2, 2, 1]
[1, 1]
[4, 3, 1, 12, 1]
[4, 3]
[4, 1, 1, 10, 3]
[4, 1]
[2, 2, 3, 9, 3]
[1, 4]
[3, 4, 3, 10, 1]
[1, 2]
[4, 4, 3, 12, 1]
[2, 3]
[3, 3, 3, 7, 1]
[4, 1]
[3, 2, 2, 9, 2]
[1, 1]
[4, 2, 3, 3, 1]
[1, 1]
[1, 

[4, 1, 2, 1, 1]
[4, 1]
[3, 2, 3, 3, 1]
[1, 1]
[2, 2, 1, 3, 3]
[2, 2]
[1, 4, 1, 4, 3]
[1, 4]
[2, 1, 2, 11, 3]
[1, 1]
[3, 1, 1, 3, 2]
[3, 1]
[2, 2, 3, 9, 1]
[1, 1]
[2, 2, 3, 9, 3]
[1, 4]
[4, 1, 1, 10, 3]
[4, 1]
[1, 2, 3, 3, 1]
[1, 1]
[2, 4, 1, 7, 3]
[1, 1]
[4, 3, 1, 12, 3]
[1, 1]
[2, 4, 3, 1, 3]
[1, 1]
[2, 3, 3, 1, 3]
[2, 3]
[1, 3, 3, 6, 3]
[1, 3]
[3, 2, 3, 3, 1]
[3, 2]
[1, 2, 2, 3, 1]
[1, 2]
[3, 1, 2, 6, 1]
[1, 1]
[4, 1, 1, 10, 3]
[4, 1]
[2, 3, 2, 2, 1]
[1, 1]
[4, 1, 2, 1, 1]
[1, 1]
[3, 1, 2, 6, 1]
[3, 1]
[3, 4, 1, 9, 2]
[1, 1]
[1, 1, 2, 1, 1]
[1, 1]
[4, 4, 3, 12, 1]
[2, 3]
[4, 1, 1, 10, 3]
[4, 1]
[3, 4, 1, 9, 3]
[3, 4]
[1, 4, 3, 6, 3]
[1, 4]
[3, 4, 1, 9, 2]
[1, 1]
[3, 2, 2, 9, 2]
[1, 1]
[1, 1, 3, 4, 1]
[1, 1]
[3, 3, 2, 1, 1]
[3, 3]
[1, 2, 3, 3, 1]
[1, 1]
[4, 1, 1, 10, 1]
[4, 1]
[2, 2, 1, 3, 2]
[1, 1]
[2, 4, 1, 7, 3]
[1, 1]
[3, 3, 2, 1, 1]
[1, 1]
[4, 3, 1, 12, 1]
[4, 3]
[1, 3, 2, 8, 2]
[1, 3]
[1, 2, 2, 3, 2]
[1, 1]
[4, 3, 3, 10, 1]
[4, 3]
[2, 2, 2, 8, 1]
[1, 1]
[1, 4, 1, 4, 1]
[1, 1]
[2

[3, 1, 1, 3, 3]
[1, 1]
[2, 4, 1, 7, 1]
[1, 1]
[3, 4, 2, 9, 2]
[3, 4]
[2, 3, 3, 1, 3]
[2, 3]
[2, 3, 3, 1, 3]
[1, 1]
[3, 3, 1, 1, 1]
[3, 3]
[4, 3, 3, 10, 1]
[4, 3]
[3, 3, 1, 1, 1]
[3, 3]
[4, 1, 3, 7, 1]
[4, 1]
[4, 3, 1, 12, 3]
[1, 1]
[4, 2, 3, 3, 1]
[4, 2]
[3, 1, 2, 6, 1]
[1, 1]
[1, 3, 1, 3, 3]
[1, 1]
[1, 3, 2, 8, 1]
[1, 1]
[3, 3, 3, 7, 1]
[4, 1]
[3, 1, 3, 1, 3]
[3, 1]
[4, 4, 1, 1, 3]
[1, 1]
[4, 4, 3, 12, 1]
[2, 3]
[1, 2, 2, 3, 2]
[1, 1]
[2, 2, 1, 3, 3]
[2, 2]
[1, 2, 3, 3, 1]
[1, 1]
[3, 1, 2, 6, 1]
[3, 1]
[4, 1, 2, 1, 1]
[4, 1]
[4, 2, 2, 2, 1]
[1, 1]
[1, 1, 2, 1, 1]
[1, 1]
[3, 4, 3, 10, 1]
[1, 2]
[2, 2, 2, 8, 1]
[1, 1]
[3, 1, 3, 1, 3]
[2, 4]
[2, 3, 3, 1, 3]
[2, 3]
[4, 1, 1, 10, 3]
[4, 1]
[3, 4, 1, 9, 3]
[3, 4]
[1, 1, 1, 1, 3]
[1, 1]
[3, 1, 3, 1, 3]
[2, 4]
[4, 2, 1, 5, 1]
[1, 1]
[1, 3, 1, 3, 3]
[1, 1]
[4, 3, 3, 10, 2]
[4, 3]
[2, 1, 2, 11, 3]
[1, 1]
[4, 1, 3, 7, 2]
[1, 1]
[3, 1, 1, 3, 3]
[1, 1]
[1, 2, 1, 7, 3]
[1, 2]
[4, 1, 1, 10, 3]
[4, 1]
[3, 3, 3, 7, 1]
[4, 1]
[1, 4, 1, 4, 1]
[1, 1]
[1,

[1, 1]
[4, 2, 1, 5, 1]
[1, 1]
[3, 1, 1, 3, 3]
[1, 1]
[1, 3, 2, 8, 1]
[1, 1]
[3, 4, 3, 10, 2]
[1, 2]
[1, 4, 3, 6, 3]
[1, 4]
[4, 2, 1, 5, 2]
[4, 2]
[3, 2, 2, 9, 2]
[1, 1]
[2, 2, 2, 8, 2]
[2, 2]
[1, 4, 3, 6, 3]
[1, 4]
[3, 4, 1, 9, 3]
[3, 4]
[1, 2, 2, 3, 2]
[1, 1]
[2, 4, 3, 1, 3]
[1, 1]
[2, 3, 3, 1, 3]
[2, 3]
[1, 4, 3, 6, 3]
[1, 4]
[1, 2, 3, 3, 1]
[1, 1]
[2, 2, 1, 3, 3]
[2, 2]
[3, 2, 3, 3, 1]
[1, 1]
[3, 2, 1, 3, 2]
[1, 1]
[3, 3, 1, 1, 1]
[3, 3]
[3, 2, 3, 3, 1]
[1, 1]
[2, 1, 1, 8, 1]
[1, 1]
[2, 4, 1, 7, 3]
[1, 1]
[1, 3, 1, 3, 3]
[1, 1]
[3, 2, 3, 3, 1]
[1, 1]
[1, 3, 1, 3, 2]
[1, 3]
[1, 2, 3, 3, 1]
[1, 1]
[2, 4, 3, 1, 3]
[1, 1]
[4, 3, 1, 12, 1]
[4, 3]
[3, 3, 1, 1, 3]
[1, 1]
[3, 3, 1, 1, 1]
[3, 3]
[2, 4, 2, 2, 1]
[2, 4]
[1, 1, 2, 1, 1]
[1, 1]
[4, 4, 3, 12, 2]
[4, 4]
[4, 2, 3, 3, 1]
[4, 2]
[2, 4, 2, 2, 1]
[2, 4]
[3, 3, 3, 7, 2]
[3, 3]
[1, 4, 3, 6, 3]
[1, 4]
[1, 1, 2, 1, 1]
[1, 1]
[1, 4, 2, 3, 2]
[1, 1]
[2, 1, 3, 1, 3]
[3, 1]
[3, 4, 1, 9, 3]
[3, 4]
[3, 4, 1, 9, 2]
[1, 1]
[4, 3, 1, 12, 1]
[4, 3]


[1, 1]
[3, 2, 1, 3, 3]
[1, 1]
[2, 3, 2, 2, 1]
[1, 1]
[1, 2, 2, 3, 1]
[1, 2]
[2, 4, 3, 1, 3]
[1, 1]
[1, 1, 1, 1, 1]
[1, 1]
[1, 1, 3, 4, 3]
[1, 1]
[1, 1, 1, 1, 1]
[1, 1]
[4, 4, 3, 12, 2]
[4, 4]
[4, 4, 1, 1, 1]
[1, 1]
[2, 2, 2, 8, 1]
[1, 1]
[2, 4, 3, 1, 3]
[1, 1]
[1, 2, 2, 3, 2]
[1, 1]
[2, 1, 1, 8, 1]
[1, 1]
[2, 4, 3, 1, 3]
[2, 4]
[1, 2, 1, 7, 1]
[1, 1]
[1, 1, 2, 1, 1]
[1, 1]
[3, 1, 3, 1, 3]
[3, 1]
[4, 2, 3, 3, 1]
[1, 1]
[4, 2, 1, 5, 1]
[1, 1]
[1, 1, 2, 1, 1]
[1, 1]
[1, 4, 3, 6, 1]
[1, 1]
[1, 2, 3, 3, 1]
[1, 1]
[4, 1, 1, 10, 1]
[4, 1]
[4, 1, 3, 7, 2]
[1, 1]
[4, 3, 3, 10, 1]
[4, 3]
[1, 1, 2, 1, 1]
[1, 1]
[2, 2, 2, 8, 1]
[1, 1]
[1, 3, 3, 6, 1]
[1, 3]
[4, 4, 3, 12, 2]
[4, 4]
[4, 3, 3, 10, 1]
[4, 3]
[3, 1, 2, 6, 1]
[1, 1]
[3, 3, 2, 1, 1]
[1, 1]
[3, 3, 2, 1, 1]
[3, 3]
[3, 2, 3, 3, 1]
[1, 1]
[1, 4, 2, 3, 1]
[1, 2]
[1, 3, 3, 6, 3]
[1, 3]
[4, 2, 3, 3, 1]
[1, 1]
[1, 2, 3, 3, 1]
[1, 1]
[1, 4, 1, 4, 1]
[1, 1]
[1, 3, 3, 6, 3]
[1, 3]
[3, 1, 3, 1, 3]
[2, 4]
[2, 3, 3, 1, 3]
[1, 1]
[2, 1, 1, 8, 1]
[1, 1]